# Import Libraries

In [2]:
import cv2
import numpy as np
# from imutils import contours
import pytesseract
import glob 
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract"
import os 
import re
import editdistance
import matplotlib.pyplot as plt 
from easyocr import Reader
from scipy.stats import shapiro
from scipy.stats import binom
%matplotlib

Using matplotlib backend: Qt5Agg


# Helper Functions

In [3]:
def showImage(text,image):
    cv2.imshow(text,image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def calculate_error(output,filePath,ext):
    fileName = os.path.basename(filePath)
    print(fileName)
    label = re.search(f"-(.*).{ext}",fileName).group(1)
    error = editdistance.distance(label,output)
    print("lable:"+label+"=>"+"output:"+output+"=>Error:"+str(error))
    print("-------------------------------------------------------------------------------------------")
    
    return label,output,error

# PreProcessing and Tesseract Functions

In [4]:
def clean_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    filtered = cv2.GaussianBlur(gray,(3,3),-1,-1)
    thresh = cv2.threshold(filtered, 0, 90, cv2.THRESH_BINARY_INV+ cv2.THRESH_OTSU)[1]
#     150
    return thresh

def cleanup_text(text):
    if text!="":
        text = text.replace(" ","")
        text = text.replace("\n","")
        return "".join([c if ord(c) < 128 else "" for c in text]).strip()
    else:
        return ""
def build_tesseract_options(psm=7):
    # tell Tesseract to only OCR alphanumeric characters
    alphanumeric = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    options = "--oem 3 -c tessedit_char_whitelist={}".format(alphanumeric)
    # set the PSM mode
    options += " --psm {}".format(psm)
    # return the built options string
    return options
def find_and_ocr(image, psm=7):
    # OCR the license plate
    options = build_tesseract_options(psm=psm)
    lpText = pytesseract.image_to_string(image, config=options)
    return lpText

# Testing

In [ ]:
folder = r"path to the folder that contains the LPR images u want to test/*"
files_to_test = []
errors = []
for file in glob.glob(folder_4)  :
    if os.path.basename(file)[0]=="h" or True: 
        if ".jpg" in file :
            image = cv2.imread(file)
            f_img = clean_image(image)
#             showImage("cleaned",f_img)
            text = cleanup_text(find_and_ocr(f_img,11))
            label,output,error = calculate_error(text,file,"jpg")
            if error == 0 :
                files_to_test.append(file)
            errors.append(error)
#                 if text!="" or text!=" ":
#                     files_to_test.append(os.path.basename(file))